In [1]:
import numpy as np
import biotite.structure as struc
from biotite.structure.io.pdb import PDBFile

In [112]:
pdb = PDBFile.read('../data/dompdb/3nskB00.pdb')
structure = pdb.get_structure(model=1)

/data/project/dohoon/miniconda3/envs/torch/lib/python3.10/site-packages/biotite/structure/io/pdb/file.py:484: UserWarning: 723 elements were guessed from atom_name.
  warnings.warn(


## Generating k-NN graph

In [113]:
from scipy.spatial.distance import cdist

def knn_edge_index(structure, k=30):
    ca_coords = np.array([a.coord for a in structure if a.atom_name == 'CA'])
    pdist = cdist(ca_coords, ca_coords, metric='euclidean')

    topk_indices = pdist.argsort(axis=1)[:, 1:k+1]
    edge_idx = np.array([[u, v] for u, neighbors in enumerate(topk_indices) for v in neighbors]).T

    return edge_idx

In [114]:
edge_idx = knn_edge_index(structure, k=30)
edge_idx

array([[ 0,  0,  0, ..., 91, 91, 91],
       [ 1,  3,  2, ..., 75, 76, 43]])

## Backbone frame

In [115]:
def to_four_atom_coordinates(atoms):
    coords = np.array([a.coord for a in atoms])
    return coords.reshape(-1, 4, 3)

four_atoms = [a for a in structure if a.atom_name in ['N', 'CA', 'C', 'O']]
four_atom_coords = to_four_atom_coordinates(four_atoms) # (#res, 4, 3)

N_IDX, CA_IDX, C_IDX, O_IDX = 0, 1, 2, 3

In [6]:
u = four_atom_coords[:, CA_IDX] - four_atom_coords[:, N_IDX]
v = four_atom_coords[:, C_IDX] - four_atom_coords[:, CA_IDX]

b = (u - v) / np.linalg.norm((u - v), axis=-1, keepdims=True)

n = np.cross(u, v)
n = n / np.linalg.norm(n, axis=-1, keepdims=True)

q = np.concatenate([
    b[:, :, None],
    n[:, :, None],
    np.cross(b, n)[:, :, None],
], axis=-1)

## Node features

### Distance features

In [8]:
def rbf(dist, d_min=0, d_max=20, d_count=16):
    d_mu = np.linspace(d_min, d_max, d_count).reshape(1, 1, 1, -1)
    d_sigma = (d_max - d_min) / d_count
    dist = dist[:, :, :, None]

    return np.exp(-(dist - d_mu)**2 / (2 * d_sigma**2))

four_atoms = [a for a in structure if a.atom_name in ['N', 'CA', 'C', 'O']]
four_atom_coords = to_four_atom_coordinates(four_atoms) # (#res, 4, 3)

dist = np.sqrt( ( (four_atom_coords[:, None, :, :] - four_atom_coords[:, :, None, :])**2 ).sum(axis=-1) )

triu_indices = [1, 2, 3, 6, 7, 11]
node_dist_feat = rbf(dist).reshape(-1, 4*4, 16)
node_dist_feat = node_dist_feat[:, triu_indices, :].reshape(-1, 6 * 16)

node_dist_feat.shape

(63, 96)

### Angle features

In [73]:
phi, psi, omega = np.nan_to_num( struc.dihedral_backbone(structure), 0.0)

# angles
backbone = structure[struc.filter_backbone(structure)]
n = len(backbone)

triplet_indices = np.array([
    np.arange(n-2),
    np.arange(1, n-1),
    np.arange(2, n)
]).T

theta1 = struc.index_angle(backbone, triplet_indices[range(0, n-2, 3)])
theta2 = struc.index_angle(backbone, triplet_indices[range(1, n-2, 3)])
theta3 = struc.index_angle(backbone, triplet_indices[range(2, n-2, 3)])

node_angle_feat = np.array([
    phi,
    psi,
    omega,
    theta1,
    np.hstack([theta2, 0.0]), # theta2 is not defined for the last residue
    np.hstack([theta3, 0.0]), # theta3 is not defined for the last residue
]).T

node_angle_feat = np.concatenate([ np.cos(node_angle_feat), np.sin(node_angle_feat) ], axis=1)
node_angle_feat.shape

(63, 12)

### Direction features

In [68]:
node_direction_feat = []

for atom_idx in [N_IDX, C_IDX, O_IDX]:
    vec_to_ca = four_atom_coords[:, atom_idx] - four_atom_coords[:, CA_IDX]
    vec_to_ca = vec_to_ca / np.linalg.norm(vec_to_ca, axis=1, keepdims=True) # Normalize
    
    direction_feature = (q.transpose(0, 2, 1) @ vec_to_ca[:, :, None]).squeeze(-1)
    node_direction_feat.append( direction_feature )
    
node_direction_feat = np.concatenate(node_direction_feat, axis=-1)
node_direction_feat.shape

(63, 9)

## Edge features

### Distance features

In [11]:
four_atoms = [a for a in structure if a.atom_name in ['N', 'CA', 'C', 'O']]
four_atom_coords = to_four_atom_coordinates(four_atoms) # (#res, 4, 3)
src_idx, dst_idx = edge_idx[0], edge_idx[1]
four_atom_coords_i, four_atom_coords_j = four_atom_coords[src_idx], four_atom_coords[dst_idx]
dist = np.sqrt( ( (four_atom_coords_i[:, None, :, :] - four_atom_coords_j[:, :, None, :])**2 ).sum(axis=-1) )

dist = rbf(dist)

dist = dist.reshape(len(dist), -1)

edge_dist_feat = dist
edge_dist_feat

(1890, 256)

### Angle features

In [28]:
def rotmat_to_quat(r):
    """Returns quarternion (x, y, z, w) converted from rotation matrix R

    R : (n, 3, 3) tensor
    returns : (n, 4) tensor
    """
    
    xx, yy, zz = r[:, 0, 0], r[:, 1, 1], r[:, 2, 2]

    q = 0.5 * np.sqrt(np.abs(1 + np.vstack([
        + xx - yy - zz,
        - xx + yy - zz,
        - xx - yy + zz,
        + xx + yy + zz,
    ])))

    sign = np.sign([
        r[:, 2, 1] - r[:, 1, 2],
        r[:, 0, 2] - r[:, 2, 0],
        r[:, 1, 0] - r[:, 0, 1],
        np.ones(len(r)),
    ])

    q = (sign * q).T
    q = q / np.linalg.norm(q, axis=1, keepdims=True)

    return q

In [13]:
np.random.seed(42)
r = np.array([
    [[0.36, 0.48, -0.80], [-0.80, 0.60, 0.00], [0.48, 0.64, 0.60]],
    [[1, 0, 0], [0, np.cos(np.pi/6), np.sin(np.pi/6)], [0, -np.sin(np.pi/6), np.cos(np.pi/6)]]
])

my_q = rotmat_to_quat(r)
my_q

array([[ 0.2       , -0.4       , -0.4       ,  0.8       ],
       [-0.25881905,  0.        ,  0.        ,  0.96592583]])

In [14]:
# For reference
from scipy.spatial.transform import Rotation as R
R.from_matrix(r).as_quat()

array([[ 0.2       , -0.4       , -0.4       ,  0.8       ],
       [-0.25881905,  0.        ,  0.        ,  0.96592583]])

In [36]:
qi, qj = q[edge_idx[0]], q[edge_idx[1]]
quat = rotmat_to_quat(qi.transpose(0, 2, 1) @ qj) # x, y, z, w

edge_angle_feat = quat
edge_angle_feat

array([[-0.04731181,  0.09597716, -0.9937651 ,  0.0313193 ],
       [-0.50313843, -0.07937256,  0.76268471,  0.39857716],
       [ 0.78711044,  0.15040974,  0.3231289 ,  0.50341015],
       ...,
       [ 0.61486383,  0.07530027, -0.34716595,  0.70409384],
       [-0.34803187, -0.38594466, -0.66715424,  0.53369068],
       [ 0.84844356, -0.11054408, -0.34265414,  0.38795834]])

In [105]:
tmp = struc.get_residues(structure)

In [110]:
tmp

(array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]),
 array(['LYS', 'THR', 'GLU', 'TRP', 'PRO', 'GLU', 'LEU', 'VAL', 'GLY',
        'LYS', 'SER', 'VAL', 'GLU', 'GLU', 'ALA', 'LYS', 'LYS', 'VAL',
        'ILE', 'LEU', 'GLN', 'ASP', 'LYS', 'PRO', 'GLU', 'ALA', 'GLN',
        'ILE', 'ILE', 'VAL', 'LEU', 'PRO', 'VAL', 'GLY', 'THR', 'ILE',
        'VAL', 'THR', 'PRO', 'GLU', 'TYR', 'ARG', 'ILE', 'ASP', 'ARG',
        'VAL', 'ARG', 'LEU', 'PHE', 'VAL', 'ASP', 'LYS', 'LEU', 'ASP',
        'ASN', 'ILE', 'ALA', 'GLU', 'VAL', 'PRO', 'ARG', 'VAL', 'GLY'],
       dtype='<U3'))

### Direction feature

In [69]:
src_idx, dst_idx = edge_idx[0], edge_idx[1]

qi = q[src_idx]
four_atom_coords_i, four_atom_coords_j = four_atom_coords[src_idx], four_atom_coords[dst_idx]

edge_direction_feat = []

for atom_idx in [N_IDX, C_IDX, O_IDX]:
    vec_to_ca = four_atom_coords_j[:, atom_idx] - four_atom_coords_i[:, CA_IDX]
    vec_to_ca = vec_to_ca / np.linalg.norm(vec_to_ca, axis=1, keepdims=True) # Normalize
    
    direction_feature = (qi.transpose(0, 2, 1) @ vec_to_ca[:, :, None]).squeeze(-1)
    edge_direction_feat.append( direction_feature )
    
edge_direction_feat = np.concatenate(edge_direction_feat, axis=-1)
edge_direction_feat.shape

(1890, 9)